# Enrich Standard Geographies

Standard geographies are jurisdictional areas determined by government agencies. At the highest level these are the countries of the world. Within these countires the heirarchial levels have different names. If you already have a list of jurisdictional area identifiers such as postal (ZIP) codes or US Census Block Group Identifiers, these can be used directly as input to the `enrich` method to retrieve demographic information about these jurisdictional areas for analysis.

## Example Use Case - Variable Variance

Through installable Python modules, there are a multitude of methods availble to select the best variables for modeling. However, for straightforward variable selection, just identifying variables with the highest variance, this is available right in Python's `statistics` module, and can be used for identifying variables with the highest variance within an jurisdictional area using smaller standard geographies.

In the United States, US Census Block Groups roughly equate to neighborhoods, and are the smallest areas with all the demographic variables available. We can use a list of these identifiers direclty with the `enrich` method to narrow down the process of identifying variables with higher variance for follow on analysis. Especially when interested in what differences exist within a jurisdictional area such as a city or county, this is a valuable first step in analysis to provide for more informed decision making for public policy.

Based on this, we are going to perform the first steps in analysis, retrieving data and idetifying high variance variables.

### Create a Country

Just as in the previous example Geoenrichment notebooks, our analysis starts with identifying the country we are going to be working with and instantating an `arcgis.geoenrichment.Country` object referencing an `arcgis.gis.GIS` source to use for analysis.

In [77]:
from arcgis.geoenrichment import Country
from arcgis.gis import GIS

usa = Country('usa', gis=GIS('pro'))

usa

<Country - United States 2021 ('local')>

### Selecting Data to Start

With nearly 20,000 demographic varialbles available, we can take advantage of data collections to narrow our variable search to at least relevant varaibles to begin with. Using the `unique` Pandas Series method, we can retrieve the data collections available and select from these to begin our search.

In [57]:
usa.enrich_variables.data_collection.unique()

array(['AgeDependency', 'homevalue', 'HistoricalHouseholds', 'Health',
       'ACS_Housing_Summary_rep', 'travelMPI', 'disability',
       'urbanizationgroupsNEW', 'Age_by_Sex_by_Race_Profile_rep',
       '1yearincrements', 'agebyracebysex', 'ClothingShoesAccessories',
       '5yearincrements', 'ACS_Population_Summary_rep', 'hispanicorigin',
       'Age', 'HistoricalHousing', 'industrybysiccode',
       'Age_50_Profile_rep', 'DaytimePopulation',
       'Age_by_Sex_Profile_rep', 'AtRisk',
       'AutomobilesAutomotiveProducts', 'Generations',
       'Automotive_Aftermarket_Expenditures_rep', 'BabyProductsToysGames',
       'basicFactsForMobileApps', 'Policy', 'householdincome',
       'tapestryhouseholdsNEW', 'businesses',
       'CivicActivitiesPoliticalAffiliation', 'classofworker', 'clothing',
       'gender', 'Community_Profile_rep', 'households', 'commute',
       'Demographic_and_Income_Comparison_Profile_rep',
       'Demographic_and_Income_Profile_rep', 'DetailedEthnicity',
    

Using Pandas filtering, we can retrieve a subset of variables for the current year (`cy`) from a selection of the data colletions listed above. Since many of the variables are in multiple data collections, we also take advantage of Pandas' ability to remove duplicates so we only have one instance of each respective variable.

In [98]:
enrich_vars = usa.enrich_variables[
    (usa.enrich_variables.name.str.lower().str.contains('cy'))
    & (
        (usa.enrich_variables.data_collection == 'occupation')
        | (usa.enrich_variables.data_collection == 'Wealth')
        | (usa.enrich_variables.data_collection == 'financial')
        | (usa.enrich_variables.data_collection == 'educationalattainment')
        | (usa.enrich_variables.data_collection == 'language')
        | (usa.enrich_variables.data_collection == 'healthinsurancecoverage')
        | (usa.enrich_variables.data_collection == 'veterans')
        | (usa.enrich_variables.data_collection == 'yearmovedin')
        | (usa.enrich_variables.data_collection == 'yearbuilt')
        | (usa.enrich_variables.data_collection == 'population')
        | (usa.enrich_variables.data_collection == 'occupation')
        | (usa.enrich_variables.data_collection == 'housingcosts')
    )
].drop_duplicates('name').reset_index(drop=True)

enrich_vars

,name,alias,data_collection,enrich_name,enrich_field_name
0,NOHS_CY,2021 Pop Age 25+: < 9th Grade,educationalattainment,educationalattainment.NOHS_CY,educationalattainment_NOHS_CY
1,SOMEHS_CY,2021 Pop Age 25+: High School/No Diploma,educationalattainment,educationalattainment.SOMEHS_CY,educationalattainment_SOMEHS_CY
2,HSGRAD_CY,2021 Pop Age 25+: High School Diploma,educationalattainment,educationalattainment.HSGRAD_CY,educationalattainment_HSGRAD_CY
3,GED_CY,2021 Pop Age 25+: GED,educationalattainment,educationalattainment.GED_CY,educationalattainment_GED_CY
4,SMCOLL_CY,2021 Pop Age 25+: Some College/No Degree,educationalattainment,educationalattainment.SMCOLL_CY,educationalattainment_SMCOLL_CY
...,...,...,...,...,...
92,VAL1M_CY,2021 Home Value $1 Million-1499999,Wealth,Wealth.VAL1M_CY,Wealth_VAL1M_CY
93,MEDVAL_CY,2021 Median Home Value,Wealth,Wealth.MEDVAL_CY,Wealth_MEDVAL_CY
94,AVGVAL_CY,2021 Average Home Value,Wealth,Wealth.AVGVAL_CY,Wealth_AVGVAL_CY
95,VALBASE_CY,2021 Home Value Base,Wealth,Wealth.VALBASE_CY,Wealth_VALBASE_CY


### Get the Geographic Level

Although we know we are going to be working with US Census Block Groups, we *do not* yet know how to let the enrich method know this. Thankfully, this is what the `levels` introspection property is for. This property returns a Pandas data frame of all the available standard geographic levels available in the country. Values from teh `level_name` column can be used as valid input for the `enrich` method's `standard_geography_level` parameter.

In [100]:
usa.levels

,level_name,alias,level_id,id_field,name_field,singular_name,plural_name,admin_level
0,block_groups,Block Groups,US.BlockGroups,ID,NAME,Block Group,Block Groups,Admin11
1,tracts,Census Tracts,US.Tracts,ID,NAME,Census Tract,Census Tracts,Admin10
2,places,Cities and Towns (Places),US.Places,ID,NAME,Place,Places,Admin9
3,zip5,ZIP Codes,US.ZIP5,ID,NAME,ZIP Code,ZIP Codes,Admin4
4,csd,County Subdivisions,US.CSD,ID,NAME,County Subdivision,County Subdivisions,Admin7
5,counties,Counties,US.Counties,ID,NAME,County,Counties,Admin3
6,cbsa,CBSAs,US.CBSA,ID,NAME,CBSA,CBSAs,Admin5
7,cd,Congressional Districts,US.CD,ID,NAME,Congressional District,Congressional Districts,Admin8
8,dma,DMAs,US.DMA,ID,NAME,DMA,DMAs,Admin6
9,states,States,US.States,ID,NAME,State,States,Admin2


## Enrich

A list of US Census Block Group identifiers with the string `block_groups` identified above through introspection can now be used to retrieve over 100 demographic variables to evaluate.

In [99]:
sea_block_group_ids = ['530330121002', '530330119002', '530330119003', '530330119004', '530330119005', '530330002003', '530330002005', '530330003001', '530330003002', '530330004011', '530330004012', '530330004013', '530330004021', '530330004022', '530330004023', '530330004024', '530330005001', '530330005002', '530330005003', '530330006001', '530330006002', '530330006003', '530330006004', '530330006005', '530330006006', '530330012001', '530330012002', '530330012003', '530330012004', '530330012005', '530330013001', '530330013002', '530330013003', '530330014001', '530330014002', '530330014003', '530330014004', '530330015001', '530330015002', '530330016001', '530330016002', '530330016003', '530330017011', '530330017012', '530330017013', '530330017021', '530330017022', '530330017023', '530330017024', '530330018001', '530330018002', '530330018003', '530330019001', '530330019002', '530330019003', '530330020002', '530330020003', '530330026002', '530330026003', '530330026004', '530330027001', '530330027002', '530330027003', '530330027004', '530330028001', '530330028002', '530330028003', '530330028004', '530330029001', '530330029002', '530330029003', '530330030001', '530330030002', '530330030003', '530330030004', '530330031001', '530330031002', '530330031003', '530330031004', '530330031005', '530330032001', '530330032002', '530330032003', '530330032004', '530330032005', '530330032006', '530330032007', '530330033001', '530330033002', '530330033003', '530330033004', '530330033005', '530330034001', '530330034002', '530330034003', '530330035001', '530330035002', '530330035003', '530330036001', '530330036002', '530330036003', '530330036004', '530330044002', '530330044004', '530330045001', '530330045002', '530330046001', '530330046002', '530330046003', '530330047001', '530330047002', '530330047003', '530330047004', '530330048001', '530330048002', '530330048003', '530330048004', '530330049001', '530330049002', '530330049003', '530330049004', '530330049005', '530330050001', '530330050002', '530330050003', '530330051001', '530330051002', '530330051003', '530330052001', '530330052002', '530330052003', '530330052004', '530330052005', '530330053012', '530330053013', '530330053014', '530330053021', '530330053022', '530330054001', '530330054002', '530330054003', '530330054004', '530330056001', '530330056002', '530330056003', '530330056004', '530330056005', '530330057001', '530330057002', '530330057003', '530330057004', '530330057005', '530330057006', '530330058011', '530330058012', '530330058013', '530330058014', '530330058021', '530330058022', '530330058023', '530330059001', '530330059002', '530330059003', '530330059004', '530330059005', '530330060001', '530330060002', '530330060003', '530330060004', '530330061001', '530330061002', '530330061003', '530330061004', '530330065001', '530330065002', '530330065003', '530330066001', '530330066002', '530330067001', '530330067002', '530330067003', '530330067004', '530330068001', '530330068002', '530330069001', '530330069002', '530330069003', '530330069004', '530330070001', '530330070002', '530330070003', '530330070004', '530330070005', '530330070006', '530330071001', '530330071002', '530330072001', '530330072002', '530330072003', '530330073001', '530330073002', '530330073003', '530330074011', '530330074012', '530330074013', '530330074014', '530330074021', '530330074022', '530330074023', '530330075001', '530330075002', '530330075003', '530330075004', '530330075005', '530330080011', '530330080012', '530330080013', '530330080014', '530330080015', '530330080021', '530330080022', '530330081001', '530330081002', '530330081003', '530330082001', '530330082002', '530330082003', '530330083001', '530330083002', '530330084001', '530330084002', '530330084003', '530330085001', '530330085002', '530330085003', '530330086001', '530330086002', '530330086003', '530330091001', '530330091002', '530330092001', '530330092002', '530330093001', '530330093002', '530330093003', '530330094001', '530330094004', '530330094005', '530330096001', '530330096002', '530330096003', '530330096004', '530330096005', '530330097011', '530330097012', '530330097013', '530330097014', '530330097021', '530330097022', '530330097023', '530330097024', '530330097025', '530330098001', '530330098002', '530330098003', '530330098004', '530330098005', '530330099001', '530330099002', '530330099003', '530330099004', '530330100021', '530330104021', '530330104024', '530330105001', '530330105002', '530330105003', '530330105004', '530330105005', '530330106001', '530330106002', '530330106003', '530330106004', '530330106005', '530330106006', '530330107011', '530330107012', '530330107021', '530330107022', '530330107023', '530330108001', '530330108002', '530330108003', '530330109001', '530330109002', '530330112001', '530330112002', '530330112003', '530330113001', '530330113002', '530330113003', '530330113004', '530330113005', '530330114011', '530330114012', '530330114013', '530330114021', '530330114022', '530330114023', '530330115001', '530330115002', '530330115003', '530330116001', '530330116002', '530330116003', '530330116004', '530330116005', '530330116006', '530330120001', '530330120002', '530330120003', '530330121001', '530330265001', '530330001001', '530330001002', '530330001003', '530330001004', '530330001005', '530330002001', '530330002002', '530330002004', '530330002006', '530330007001', '530330007002', '530330007003', '530330007004', '530330008001', '530330008002', '530330009001', '530330009002', '530330010001', '530330010002', '530330011001', '530330011002', '530330020001', '530330021001', '530330021002', '530330021003', '530330022001', '530330022002', '530330022003', '530330022004', '530330024001', '530330024002', '530330024003', '530330025001', '530330025002', '530330026001', '530330038001', '530330038002', '530330039001', '530330039002', '530330040001', '530330040002', '530330041001', '530330041002', '530330041003', '530330041004', '530330041005', '530330041006', '530330042001', '530330042002', '530330042003', '530330042004', '530330042005', '530330043011', '530330043012', '530330043013', '530330043021', '530330043022', '530330044001', '530330044003', '530330053011', '530330053023', '530330062001', '530330062002', '530330062003', '530330063001', '530330063002', '530330063003', '530330063004', '530330064001', '530330064002', '530330064003', '530330076001', '530330076002', '530330076003', '530330077001', '530330077002', '530330077003', '530330077004', '530330078001', '530330078002', '530330078003', '530330078004', '530330079001', '530330079002', '530330079003', '530330079004', '530330079005', '530330087001', '530330087002', '530330087003', '530330088001', '530330088002', '530330088003', '530330089001', '530330089002', '530330089003', '530330089004', '530330090001', '530330090002', '530330094002', '530330094003', '530330095001', '530330095002', '530330095003', '530330095004', '530330100011', '530330100012', '530330100013', '530330100022', '530330100023', '530330100024', '530330101001', '530330101002', '530330101003', '530330101004', '530330101005', '530330102001', '530330102002', '530330102003', '530330102004', '530330103001', '530330103002', '530330103003', '530330103004', '530330103005', '530330104011', '530330104012', '530330104013', '530330104022', '530330104023', '530330110011', '530330110012', '530330110013', '530330110021', '530330110022', '530330110023', '530330111011', '530330111012', '530330111013', '530330111014', '530330111021', '530330111022', '530330111023', '530330111024', '530330117001', '530330117002', '530330117003', '530330117004', '530330118001', '530330118002', '530330118003', '530330118004', '530330118005', '530330118006', '530330119001']

enrich_df = usa.enrich(sea_block_group_ids, enrich_variables=enrich_vars, standard_geography_level='block_groups')

enrich_df.info()
enrich_df.head()

## Calculate Variance

Variation can now be calculated for the retrieved variables to identify those with exceedingly high variance, and follow on analysis can be used for feature selection or feature reduction to address covariance between variables and perform modeling.

The first step, though, is reducing the data frame to just the quantitative demographic variables. This is accomplished using a list comprehension to keep just these demographic columns.

In [103]:
# get just the enrich columns
enrich_cols = [c for c in enrich_df if c in usa.enrich_variables.name.str.lower().values]

enrich_df = enrich_df.set_index('id_field').loc[:,enrich_cols]

enrich_df.info()
enrich_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 478 entries, 530330121002 to 530330119001
Data columns (total 97 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   nohs_cy     478 non-null    float64
 1   somehs_cy   478 non-null    float64
 2   hsgrad_cy   478 non-null    float64
 3   ged_cy      478 non-null    float64
 4   smcoll_cy   478 non-null    float64
 5   asscdeg_cy  478 non-null    float64
 6   bachdeg_cy  478 non-null    float64
 7   graddeg_cy  478 non-null    float64
 8   educbasecy  478 non-null    float64
 9   civlbfr_cy  478 non-null    float64
 10  occbase_cy  478 non-null    float64
 11  occmgmt_cy  478 non-null    float64
 12  occbus_cy   478 non-null    float64
 13  occcomp_cy  478 non-null    float64
 14  occarch_cy  478 non-null    float64
 15  occssci_cy  478 non-null    float64
 16  occssrv_cy  478 non-null    float64
 17  occlegl_cy  478 non-null    float64
 18  occeduc_cy  478 non-null    float64
 19  occent_cy   47

,nohs_cy,somehs_cy,hsgrad_cy,ged_cy,smcoll_cy,asscdeg_cy,bachdeg_cy,graddeg_cy,educbasecy,civlbfr_cy,occbase_cy,occmgmt_cy,occbus_cy,occcomp_cy,occarch_cy,occssci_cy,occssrv_cy,occlegl_cy,occeduc_cy,occent_cy,occhtch_cy,occhlth_cy,occprot_cy,occfood_cy,occbldg_cy,occpers_cy,occsale_cy,occadmn_cy,occfarm_cy,occcons_cy,occmain_cy,occprod_cy,occtran_cy,tothh_cy,hinc0_cy,hinc15_cy,hinc25_cy,hinc35_cy,hinc50_cy,hinc75_cy,hinc100_cy,hinc150_cy,hinc200_cy,medhinc_cy,avghinc_cy,pci_cy,agginc_cy,agghinc_cy,hincbasecy,avgia15_cy,avgia25_cy,avgia35_cy,avgia45_cy,avgia55_cy,avgia65_cy,avgia75_cy,di0_cy,di15_cy,di25_cy,di35_cy,di50_cy,di75_cy,di100_cy,di150_cy,di200_cy,meddi_cy,aggdi_cy,avgdi_cy,dibase_cy,nw0_cy,nw15_cy,nw35_cy,nw50_cy,nw75_cy,nw100_cy,nw150_cy,nw250_cy,nw500_cy,mednw_cy,avgnw_cy,aggnw_cy,nwbase_cy,val0_cy,val50k_cy,val100k_cy,val150k_cy,val200k_cy,val250k_cy,val300k_cy,val400k_cy,val500k_cy,val750k_cy,val1m_cy,medval_cy,avgval_cy,valbase_cy,wlthindxcy
id_field,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
530330121002,2.0,33.0,130.0,21.0,235.0,94.0,513.0,212.0,1240.0,1023.0,959.0,197.0,71.0,86.0,50.0,0.0,51.0,43.0,37.0,0.0,68.0,14.0,26.0,8.0,7.0,6.0,88.0,64.0,1.0,15.0,14.0,42.0,71.0,688.0,0.0,12.0,18.0,45.0,94.0,69.0,146.0,120.0,184.0,132006.0,159503.0,66196.0,109753388.0,109737756.0,688.0,64980.0,154983.0,176016.0,174324.0,175930.0,158816.0,111859.0,2.0,20.0,23.0,68.0,103.0,99.0,180.0,89.0,104.0,105107.0,82794420.0,120341.0,688.0,12.0,5.0,3.0,15.0,14.0,28.0,48.0,96.0,145.0,884070.0,3145778.0,2.164295e+09,688.0,0.0,0.0,0.0,0.0,0.0,2.0,34.0,108.0,274.0,102.0,72.0,642792.0,710982.0,601.0,262.0
530330119002,35.0,17.0,125.0,0.0,219.0,169.0,373.0,235.0,1173.0,999.0,942.0,66.0,92.0,82.0,33.0,0.0,1.0,23.0,18.0,17.0,170.0,55.0,2.0,48.0,36.0,7.0,18.0,97.0,0.0,46.0,19.0,8.0,104.0,604.0,32.0,37.0,18.0,85.0,116.0,59.0,84.0,94.0,79.0,79883.0,114931.0,44103.0,69418544.0,69418544.0,604.0,34748.0,133017.0,160358.0,121854.0,123713.0,83355.0,54447.0,40.0,38.0,48.0,99.0,104.0,64.0,124.0,47.0,40.0,66626.0,52967478.0,87695.0,604.0,68.0,20.0,10.0,25.0,21.0,32.0,79.0,131.0,82.0,309364.0,1406312.0,8.494126e+08,604.0,0.0,0.0,0.0,0.0,1.0,3.0,85.0,126.0,225.0,52.0,0.0,534444.0,556148.0,492.0,141.0
530330119003,37.0,0.0,159.0,24.0,299.0,92.0,286.0,27.0,924.0,860.0,807.0,57.0,28.0,0.0,31.0,18.0,0.0,28.0,16.0,0.0,41.0,66.0,17.0,56.0,0.0,0.0,25.0,163.0,0.0,18.0,13.0,145.0,85.0,418.0,54.0,3.0,0.0,39.0,68.0,27.0,112.0,34.0,81.0,105049.0,126803.0,40699.0,53031182.0,53003826.0,418.0,75144.0,143550.0,180664.0,149294.0,133043.0,92354.0,57994.0,55.0,2.0,16.0,52.0,58.0,69.0,84.0,39.0,43.0,82643.0,40736416.0,97456.0,418.0,64.0,13.0,7.0,17.0,18.0,16.0,29.0,73.0,67.0,372548.0,1800845.0,7.527533e+08,418.0,2.0,0.0,0.0,1.0,1.0,4.0,81.0,226.0,20.0,0.0,1.0,434956.0,432589.0,336.0,169.0
530330119004,121.0,42.0,201.0,130.0,183.0,232.0,125.0,92.0,1126.0,762.0,738.0,38.0,33.0,5.0,3.0,1.0,1.0,3.0,59.0,1.0,11.0,60.0,2.0,32.0,48.0,10.0,47.0,105.0,0.0,27.0,21.0,37.0,194.0,468.0,48.0,53.0,28.0,24.0,55.0,71.0,66.0,60.0,63.0,82477.0,108716.0,32433.0,50920582.0,50878983.0,468.0,106330.0,114966.0,183461.0,125452.0,84803.0,75305.0,75969.0,60.0,53.0,25.0,37.0,82.0,55.0,91.0,33.0,32.0,66004.0,38904089.0,83128.0,468.0,97.0,24.0,10.0,24.0,18.0,28.0,47.0,68.0,60.0,212448.0,1268547.0,5.936801e+08,468.0,0.0,0.0,0.0,1.0,2.0,3.0,89.0,108.0,144.0,0.0,1.0,473148.0,495546.0,348.0,130.0
530330119005,132.0,51.0,97.0,162.0,212.0,100.0,318.0,241.0,1313.0,1261.0,1123.0,79.0,92.0,42.0,6.0,1.0,23.0,7.0,64.0,1.0,74.0,38.0,42.0,72.0,75.0,55.0,32.0,104.0,0.0,63.0,30.0,143.0,80.0,559.0,46.0,23.0,66.0,16.0,42.0,59.0,137.0,81.0,89.0,106650.0,123549.0,37119.0,69078958.0,69064153.0,559.0,84584.0,121235.0,138280.0,141519.0,140638.0,104960.0,63104.0,51.0,50.0,39.0,29.0,64.0,92.0,139.0,45.0,50.0,85714.0,53558731.0,95812.0,559.0,89.0,21.0,13.0,26.0,24.0,40.0,69.0,74.0,81.0,244894.0,792528.0,4.430230e+08,559.0,2.0,0.0,0.0,0.0,2.0,2.0,193.0,122.0,86.0,0.0,1.0,4040

### Calculate Variance

This is a very simple way, taking advantage of Pandas, to calculate variance and select the 20 variables with the highest variance. Since, in theory, this is a complete population representation, not just a sample, we are setting the degrees of freedom (`ddof`) to zero. This process is *very* similar to just using the `SelectKBest` method in SciKit-Learn.

In [92]:
top20 = enrich_df.var(ddof=0).sort_values(ascending=False).iloc[:20]
top20.name = 'variance'

top20_df

aggnw_cy      5.532277e+17
agghinc_cy    5.229259e+15
agginc_cy     5.213711e+15
aggdi_cy      2.515085e+15
avgnw_cy      1.915711e+12
mednw_cy      1.884522e+11
medval_cy     8.777470e+10
avgval_cy     8.646778e+10
avgia45_cy    5.600466e+09
avgia55_cy    5.355705e+09
avgia35_cy    4.171154e+09
avghinc_cy    3.506248e+09
avgia65_cy    3.401690e+09
avgia25_cy    2.638046e+09
avgia75_cy    2.238534e+09
medhinc_cy    1.981167e+09
avgia15_cy    1.371848e+09
avgdi_cy      1.319724e+09
meddi_cy      1.163338e+09
pci_cy        7.353389e+08
dtype: float64

### Make it Human Readable

Since the column names are not tremendously *human* readable, we can join the `enrich_variables` data frame to the Pandas Series with variance to see what these variables represent. None of this is tremendously surprising in Seattle. With a booming technology industry and soaring real estate, income and net worth are not surprising to emerge as high variance variables.

In [94]:
ev = usa.enrich_variables
ev.index = ev.name.str.lower()
top20_df = ev.join(top20, how='right').loc[:,['name', 'alias', 'variance']]


,name,alias,variance
aggnw_cy,AGGNW_CY,2021 Aggregate Net Worth,5.520703e+17
agghinc_cy,AGGHINC_CY,2021 Aggregate HH Income,5.218319e+15
agginc_cy,AGGINC_CY,2021 Aggregate Income,5.202804e+15
aggdi_cy,AGGDI_CY,2021 Aggregate Disposable Income,2.509823e+15
avgnw_cy,AVGNW_CY,2021 Average Net Worth,1.911703e+12
mednw_cy,MEDNW_CY,2021 Median Net Worth,1.880580e+11
medval_cy,MEDVAL_CY,2021 Median Home Value,8.759107e+10
avgval_cy,AVGVAL_CY,2021 Average Home Value,8.628688e+10
avgia45_cy,AVGIA45_CY,2021 Avg HH Income: HHr 45-54,5.588749e+09
avgia55_cy,AVGIA55_CY,2021 Avg HH Income: HHr 55-64,5.344500e+09


## Continuing Analysis

From here, a variety of techniques can be used, but with so many income and net worth variables, before subsequent modeling steps, covariacne needs to be addressed. Using the Geoenrichment dramatically streamlines getting to this point, though. It provides extremely easy access to thousands of demographic variables for modeling and analysis directly in Python, making it easy to integrate with data engineering pipelines.